### Exploratory Analysis of DuckDB br_funds.db

In [ ]:
import duckdb
import pandas as pd

In [ ]:
# Connect to your local DuckDB database file
conn = duckdb.connect('../data/br_funds.db')


# List all tables in the database
tables = conn.execute("SHOW TABLES").fetchall()
print("Available tables:", tables)


# close the connection when done
conn.close()

### Analysis of assets table

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM assets LIMIT 20").fetchdf()

conn.close()

df.head()

In [ ]:
id = df["asset_id"][2]
print("Asset ID of first record:", id)

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

values = conn.execute("SELECT DISTINCT financial_instrument FROM assets").fetchall()
print("Distinct asset classes:", values)

conn.close()

### Asset Class Meanings:
- INVESTMENT_FUND: Shares or units in other mutual funds/ETFs (fund of funds)
- EQUITY: Stocks/shares in companies
- FIXED_INCOME: Bonds, treasury notes, or other debt securities
- CASH: Cash holdings, money market instruments, or cash equivalents
- DERIVATIVES: Options, futures, swaps, or other derivative contracts (used for hedging or speculation)
- UNSPECIFIED: Assets that don't fit the other categories or missing classification data

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

query = """
SELECT * 
FROM assets
WHERE asset_class = 'EQUITY'
"""

result = conn.execute(query).df()
conn.close()
result.head(10)



### Analysis of fund_performance_indicator table

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM fund_performance_indicators LIMIT 5").fetchdf()

conn.close()

df.head()

### Analysis of fund_snapshots table

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM fund_snapshots LIMIT 5").fetchdf()

conn.close()

df.head()

### Analysis of funds table (OF INTEREST)

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM funds LIMIT 5").fetchdf()

conn.close()

df.head()

In [ ]:
import duckdb

conn = duckdb.connect('../data/br_funds.db')

# Obtener conteo de fondos con cancellation_date (no nulos)
count_with_cancellation = conn.execute("""
    SELECT COUNT(*) as count
    FROM funds
    WHERE cancellation_date IS NOT NULL
""").fetchone()[0]

# Obtener conteo de fondos sin cancellation_date (nulos)
count_without_cancellation = conn.execute("""
    SELECT COUNT(*) as count
    FROM funds
    WHERE cancellation_date IS NULL
""").fetchone()[0]

conn.close()

print(f"Fondos con cancellation_date: {count_with_cancellation}")
print(f"Fondos sin cancellation_date (None): {count_without_cancellation}")
print(f"Total: {count_with_cancellation + count_without_cancellation}")

In [ ]:
import duckdb

conn = duckdb.connect('../data/br_funds.db')

# Obtener conteo agrupado por status para cada tipo de cancellation_date
df_counts = conn.execute("""
    SELECT 
        CASE 
            WHEN cancellation_date IS NOT NULL THEN 'With cancellation_date'
            ELSE 'Without cancellation_date (None)'
        END as cancellation_type,
        status,
        COUNT(*) as count
    FROM funds
    GROUP BY 
        CASE 
            WHEN cancellation_date IS NOT NULL THEN 'With cancellation_date'
            ELSE 'Without cancellation_date (None)'
        END,
        status
    ORDER BY cancellation_type, status
""").fetchdf()

conn.close()

print(df_counts)

## Funds with cancellation date

In [ ]:
import duckdb

conn = duckdb.connect('../data/br_funds.db')

# Verificar otros campos que puedan indicar si están activos
df_analysis = conn.execute("""
    SELECT 
        status,
        COUNT(*) as count,
        COUNT(net_asset_value_date) as has_nav_date,
        COUNT(activity_start_date) as has_activity_date,
        MIN(timestamp) as earliest_timestamp,
        MAX(timestamp) as latest_timestamp,
        AVG(CASE WHEN net_asset_value.value > 0 THEN 1 ELSE 0 END) as pct_with_positive_nav
    FROM funds
    WHERE cancellation_date IS NULL AND status = 'UNSPECIFIED'
    GROUP BY status
""").fetchdf()

conn.close()

print(df_analysis)

# Funds by Anbima classification

In [ ]:
import duckdb

conn = duckdb.connect('../data/br_funds.db')

# Ver todas las clasificaciones ANBIMA y sus conteos
df_anbima = conn.execute("""
    SELECT 
        anbima_classification,
        COUNT(*) as count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as percentage
    FROM funds
    WHERE cancellation_date IS NULL  -- Solo fondos no cancelados
    GROUP BY anbima_classification
    ORDER BY count DESC
""").fetchdf()

conn.close()

print(df_anbima)

In [ ]:
df.describe()

In [ ]:
print("\nColumns (one per line):")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

### Analysis of positions table

In [ ]:
conn = duckdb.connect('../data/br_funds.db')

df = conn.execute("SELECT * FROM positions LIMIT 5").fetchdf()

conn.close()

df.head()